In [13]:
##SGDP

import projmodules as pm

pd="/mnt/solexa/mel_yang/2.2_neolithic/admixture/"
sgdpfile=open(pd+"SGDP_ALL.txt",'r')
sgdp=[line.strip() for line in sgdpfile]
sgdpfile.close()

ancasn=["Tianyuan",'Longlin','Longlin_com','Xinyi',
        'Xinyi_other','Daxi','LiangDaoChineseNeolithic','Boisman_MN',
        'BoshanChineseNeolithic','Bbdong','Linzi','HDYM1','HQSDW',
        'Zongri','Pukagongma','Chokhopani1','Mebrak','Samdzong',
        'Kolyma_River',"Donghuigou","Balikun"]
ancpub=["UstIshim","Oase1","Kostenki14","GoyetQ116-1","KremsWA3",
     "Vestonice16","Yana_old","Yana_old2","Ostuni1","HohleFels49",
     "Malta1",'AfontovaGora3',"ElMiron","Villabruna","Bichon","Rochedane","Ranchot88",
     "Satsurblia","Kotias","Karelia","Motala12","Loschbour","LaBrana1",
     "Hungarian.KO1","Stuttgart","Anatolia_N","Iran_N","Levant_N","Natufian","Mota","Clovis","Saqqaq"]
inds=ancasn+ancpub+sgdp

datpd = "/mnt/solexa/mel_yang/data/2.2M/"
fh="UPA_all8"
newname = 'ADMIXALL_102417'
pm.indtoignore(inds, datpd+fh+'.ind', datpd+fh+'.'+newname+'.ind')
pm.convertf_par(datpd+fh,pd+fh+'.'+newname+'.ped','PED',pd+
                'CONVERTF.'+fh+'.'+newname+'.par',indfilename=datpd+fh+'.'+newname)


'Made par file for convertf!'

In [12]:
##HO

import projmodules as pm
import HOdata

pd="/mnt/solexa/mel_yang/2.2_neolithic/admixture/"
hodat=HOdata.pdpops

ancasn=["Tianyuan",'Longlin','Longlin_com','Xinyi','Daxi','LiangDaoChineseNeolithic','Boisman_MN',
        'BoshanChineseNeolithic','Bbdong','Linzi','HDYM1','HQSDW',
        'Zongri','Pukagongma','Chokhopani1','Mebrak','Samdzong',
        'Kolyma_River',"Donghuigou","Balikun"]
ancpub=["UstIshim","Oase1","Kostenki14","GoyetQ116-1","KremsWA3",
     "Vestonice16","Yana_old","Yana_old2","Ostuni1","HohleFels49_N",
     "Malta1",'AfontovaGora3',"ElMiron","Villabruna","Bichon","Rochedane_N","Ranchot88_N",
     "Satsurblia","Kotias","Karelia","Motala_HG","Loschbour","LaBrana1",
     "Hungarian.KO1","Stuttgart","Anatolia_N","Iran_N","Levant_N","Natufian","Mota","Clovis","Saqqaq"]
inds=ancasn+ancpub+hodat

datpd = "/mnt/solexa/mel_yang/data/HO/"
fh="ind_A_v13_TBH6"
newname = 'ADMIX_noAFR_102417'
pm.indtoignore(inds, datpd+fh+'.ind', datpd+fh+'.'+newname+'.ind')
pm.convertf_par(datpd+fh,pd+fh+'.'+newname+'.ped','PED',pd+
                'CONVERTF.'+fh+'.'+newname+'.par',indfilename=datpd+fh+'.'+newname)


'Made par file for convertf!'

In [20]:
%%bash
pd="/mnt/solexa/mel_yang/2.2_neolithic/admixture/" #NEED TO MOVE TO admixture/ FIRST
#fh="UPA_all8.ADMIX_102417"
fh="ind_A_v13_TBH6.ADMIX_noAFR_102417"
idp=".200_25_0.4"
mv ${pd}${fh}.ped.geno ${pd}${fh}.ped
mv ${pd}${fh}.ped.ind ${pd}${fh}.pedind
cut -c1-41 ${pd}${fh}.ped.snp > ${pd}${fh}.map

echo "plink --noweb --file ${pd}${fh} --make-bed --out ${pd}${fh} --geno 0.999"


##Prune dataset, using 50 SNP windows, step 10 SNPs, remove SNP pairs with R^2>0.1
echo "plink --noweb --bfile ${pd}${fh} --indep-pairwise 200 25 0.4 --out ${pd}${fh}_plink${idp}" 

##Keep set of untargeted SNPs from above
echo "plink --noweb --bfile ${pd}${fh} --extract ${pd}${fh}_plink${idp}.prune.in --make-bed --out ${pd}${fh}_pruned${idp}"  

plink --noweb --file /mnt/solexa/mel_yang/2.2_neolithic/admixture/ind_A_v13_TBH6.ADMIX_noAFR_102417 --make-bed --out /mnt/solexa/mel_yang/2.2_neolithic/admixture/ind_A_v13_TBH6.ADMIX_noAFR_102417 --geno 0.999
plink --noweb --bfile /mnt/solexa/mel_yang/2.2_neolithic/admixture/ind_A_v13_TBH6.ADMIX_noAFR_102417 --indep-pairwise 200 25 0.4 --out /mnt/solexa/mel_yang/2.2_neolithic/admixture/ind_A_v13_TBH6.ADMIX_noAFR_102417_plink.200_25_0.4
plink --noweb --bfile /mnt/solexa/mel_yang/2.2_neolithic/admixture/ind_A_v13_TBH6.ADMIX_noAFR_102417 --extract /mnt/solexa/mel_yang/2.2_neolithic/admixture/ind_A_v13_TBH6.ADMIX_noAFR_102417_plink.200_25_0.4.prune.in --make-bed --out /mnt/solexa/mel_yang/2.2_neolithic/admixture/ind_A_v13_TBH6.ADMIX_noAFR_102417_pruned.200_25_0.4


## Actual admixture runs! (w/CV)

In [28]:
%%bash
#%%writefile /mnt/solexa/mel_yang/2.2_neolithic/admixture/runadmixtureCV.sh

##also do cv at runadmixturecv.sh

pd="/mnt/solexa/mel_yang/2.2_neolithic/admixture/"
fh="ind_A_v13_TBH6.ADMIX_noAFR_102417_pruned.200_25_0.4"
for K in `seq 2 20`; do 
    echo "admixture --cv ${pd}${fh}.bed ${K} > ${pd}${fh}.admix.log${K}.out &"
    #admixture ${pd}${fh}.bed ${K} > ${pd}${fh}.admix.${K}.out &
    #admixture --cv ${pd}${fh}.bed ${K} | tee ${pd}${fh}.admix.log${K}.out
    #echo "admixture --cv ${pd}${fh}.bed ${K} > ${pd}${fh}.admix.log${K}.out"
    #echo "admixture ${pd}${fh}.bed ${K} > ${pd}${fh}.admix.${K}.out"
done

#grep -h CV ${pd}${fh}.admix.log*out > ${pd}${fh}.admix.CVerror ##get CVerrors

admixture --cv /mnt/solexa/mel_yang/2.2_neolithic/admixture/ind_A_v13_TBH6.ADMIX_noAFR_102417_pruned.200_25_0.4.bed 2 > /mnt/solexa/mel_yang/2.2_neolithic/admixture/ind_A_v13_TBH6.ADMIX_noAFR_102417_pruned.200_25_0.4.admix.log2.out &
admixture --cv /mnt/solexa/mel_yang/2.2_neolithic/admixture/ind_A_v13_TBH6.ADMIX_noAFR_102417_pruned.200_25_0.4.bed 3 > /mnt/solexa/mel_yang/2.2_neolithic/admixture/ind_A_v13_TBH6.ADMIX_noAFR_102417_pruned.200_25_0.4.admix.log3.out &
admixture --cv /mnt/solexa/mel_yang/2.2_neolithic/admixture/ind_A_v13_TBH6.ADMIX_noAFR_102417_pruned.200_25_0.4.bed 4 > /mnt/solexa/mel_yang/2.2_neolithic/admixture/ind_A_v13_TBH6.ADMIX_noAFR_102417_pruned.200_25_0.4.admix.log4.out &
admixture --cv /mnt/solexa/mel_yang/2.2_neolithic/admixture/ind_A_v13_TBH6.ADMIX_noAFR_102417_pruned.200_25_0.4.bed 5 > /mnt/solexa/mel_yang/2.2_neolithic/admixture/ind_A_v13_TBH6.ADMIX_noAFR_102417_pruned.200_25_0.4.admix.log5.out &
admixture --cv /mnt/solexa/mel_yang/2.2_neolithic/admixture/ind_

## Get CV!

In [29]:
%%bash
pd="/mnt/solexa/mel_yang/2.2_neolithic/admixture/"
fh="UPA_all8.ADMIX_102417_pruned.200_25_0.4"

grep -h CV ${pd}${fh}.admix.log*out > ${pd}${fh}.admix.CVerror ##get CVerrors

## Get set of population names to make ".wID.Q"

In [ ]:
%%bash
pd="/mnt/solexa/mel_yang/2.2_neolithic/admixture/"
datpd="/mnt/solexa/mel_yang/data/2.2M/"
fh="UPA_all8.ADMIX_102417"
grep -v Ignore ${datpd}${fh}.ind > ${pd}${fh}.noIgnore.ind

In [49]:
%%bash
pd="/mnt/solexa/mel_yang/2.2_neolithic/admixture/"
indfh="UPA_all8.ADMIX_102417"
Qfh="UPA_all8.ADMIX_102417_pruned.200_25_0.4"
K=2
for K in 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19; do 
    paste ${pd}${Qfh}.${K}.Q ${pd}${indfh}.noIgnore.ind > ${pd}${Qfh}.${K}.wID.Q
done

paste: /mnt/solexa/mel_yang/2.2_neolithic/admixture/UPA_all8.ADMIX_102417_pruned.200_25_0.4.18.Q: No such file or directory


## Admixture Plot

In [48]:
%%writefile /mnt/solexa/mel_yang/2.2_neolithic/admixture/admixtureplot.R
##MAKE ADMIXTURE PLOT

args <- commandArgs(trailingOnly = TRUE)

if(length(args) < 2) {
  args <- c("--help")
}

## Help section
if("--help" %in% args) {
  cat("Example:\n
      Rscript admixtureplot.R PD FH SORTORDER LABEL K\n
      PD=project directory\n
      FH = filehandle with ending '.wID.Q', '.pops'\n
      SORTORDER = sortorder of different K, if '0' then default order, otherwise separate by commas, index from 1, no larger than K\n
      LABEL = new name to add on\n
      K = number cluster to do\n
      EX. Rscript admixtureplot.R  /mnt/solexa/mel_yang/2.2_neolithic/admixture/ UPA_all8.ADMIX_102417_pruned.200_25_0.4 1,2,3 4\n
          Notes:\n
          (1) EX TO MAKE POP FILE: cat UPA_all8.ADMIX_102417_pruned.200_25_0.4.2.wID.Q | awk '{print $5}' | sort -u > UPA_all8.ADMIX_102417_pruned.200_25_0.4.pops\n
          (2) No more than four numbers in order.\n\n")

  q(save="no")
}



pd=args[1] #"/mnt/solexa/mel_yang/2.2_neolithic/admixture/"
fh=args[2] #"UPA_all8.ADMIX_102417_pruned.200_25_0.4"
order=strsplit(args[3],",")[[1]]
mypops=read.table(paste(pd,fh,".pops",sep=""))$V1
subset=args[4]
for (K in c(as.numeric(args[5]))) {
    tbl=read.table(paste(pd,fh,".",K,".wID.Q",sep=""))
    #tbl=tbl[with(tbl,order(tbl[,K+3],tbl[,4],tbl[,1],tbl[,2],tbl[,3])),]
    tbl=tbl[with(tbl,order(tbl[,K+3])),]
    for (i in 1:length(order)) {
        #print (as.numeric(order[i]))
        tbl=tbl[with(tbl,order(tbl[,as.numeric(order[i])])),]
        
    }
    #tbl=tbl[with(tbl,order(tbl[,4],tbl[,1],tbl[,2],tbl[,3])),]
    #if (K==4){
    tblnames=c()
    for (i in tbl[,K+3]) {
        if (i %in% mypops) {
             if (i %in% tblnames==TRUE) {
                tblnames=c(tblnames," ")
            }
            else {
                tblnames=c(tblnames,i)
            }        
        }
    } 
    mycolnames=c()
    for (i in 1:ncol(tbl)) {
        if (i<=K) { mycolnames=c(mycolnames,i)}
    }
    mycolnames=c(mycolnames,"Ind","Sex","Pop")
    colnames(tbl) = mycolnames
    #}
    #print (tbl)
    png(paste(pd,"images/",fh,".",subset,".",K,".png",sep=""),width=2400,height=600)
    subtbl=subset(tbl,Pop%in%mypops)[,1:4]
    #mynames=rep("",nrow(subtbl))
    mynames=subtbl$Pop
    
    barplot(t(as.matrix(subtbl)), col=rainbow(K),xlab="Individual #", ylab="Ancestry", border=NA,names.arg=tblnames,las=2,cex.names=0.7)
    dev.off()
    #for (mypop in mypops) {
    #	png(paste(pd,"images/",fh,".",mypop,".",K,".png",sep="")) #,width=2400,height=600)
    #	subtbl=subset(tbl,V7==mypop)[,1:4]
    #	mynames=rep("",nrow(subtbl))
    #	barplot(t(as.matrix(subtbl)), col=rainbow(K),xlab="Individual #", ylab="Ancestry", border=NA,names.arg=mynames,las=2,cex.names=0.7)
	#	dev.off()
    #}
    
}

Overwriting /mnt/solexa/mel_yang/2.2_neolithic/admixture/admixtureplot.R
